updated - run this one on localhost:8888

### Anomaly Detection in Server Logs
#### CS 5614: Big Data Engineering
#### By: Vanessa Eichensehr and Bradley Freedman


**Project Objective:**  

Build a machine learning-based model that detects anomalies on a high volume high velocity log base. 

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-kafka-streaming").\
        master("spark://spark-master:7077").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0"). \
        config("spark.executor.memory", "512m").\
        getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0c62bc65-e452-426c-b8db-d80c808eb75f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 381ms :: artifacts dl 23

**Step 1:** 
Create a streaming DataFrame in Spark that reads data from a Kafka topic named "topic_test" and starts
processing from the beginning of the topic's log using the earliest available offset. Use kafka:9093 as the bootstrap
server.

In [ ]:
df_streamed_raw = (spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9093").option("subscribe", "topic_test").load())

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

# convert byte stream to string
df_streamed_kv = (df_streamed_raw
    .withColumn("key", df_streamed_raw["key"].cast(StringType()))
    .withColumn("value", df_streamed_raw["value"].cast(StringType())))

test_query = (df_streamed_kv 
              .writeStream \
              .format("memory") # output to memory \
              .outputMode("update") # only write updated rows to the sink \
              .queryName("test_query_table")  # Name of the in memory table \
              .start())

#### If all goes well, the following cell should display a table populated with values being streamed from you Kafka producer. NOTE: If you recently ran the producer, it may take a while before the table is populated. Keep rerunning the cell to check for updates:

In [ ]:
spark.sql("select * from test_query_table").show()

In [ ]:
test_query.stop()

#### The following cells contain code that take the streamed dataframe and formats it properly into a table. If any of the given cells fails, there might be a formatting issue with one of your previous solutions. 

In [ ]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, BooleanType, LongType, IntegerType

event_schema = StructType([
    StructField("ip_address", StringType()),
    StructField("date_time", StringType()),
    StructField("request_type", StringType()),
    StructField("request_arg", StringType()),
    StructField("status_code", StringType()),
    StructField("response_size", StringType()),
    StructField("referrer", StringType()),
    StructField("user_agent", StringType())
])

# Parse the events from JSON format
df_parsed = (df_streamed_kv
           # Sets schema for event data
           .withColumn("value", from_json("value", event_schema))
          )

In [ ]:
df_formatted = (df_parsed.select(
    col("key").alias("event_key")
    ,col("topic").alias("event_topic")
    ,col("timestamp").alias("event_timestamp")
    ,col("value.ip_address").alias("ip_address")
    ,col("value.date_time").alias("date_time")
    ,col("value.request_type").alias("request_type")
    ,col("value.request_arg").alias("request_arg")
    ,col("value.status_code").alias("status_code")
    ,col("value.response_size").cast(IntegerType()).alias("response_size")
    ,col("value.referrer").alias("referrer")
    ,col("value.user_agent").alias("user_agent")
))

In [ ]:
# Write the parsed data to console
query = (df_formatted.writeStream.format("console").outputMode("append").trigger(processingTime='5 seconds').start())

In [ ]:
# Print the name of active streams (This may be useful during debugging)
for s in spark.streams.active:
    print(f"ID:{s.id} | NAME:{s.name}")

In [ ]:
query.stop()

**Best Guess at Next Steps**

Load labeled training data

In [63]:
training_df = spark.read.csv("/data/synthetic_with_anomalies_NEW.csv", header=True, inferSchema=True)
training_df.show(5)

+-------------+--------------------+------+--------------------+------+------+--------------------+--------------------+---------+-------------+
|           ip|                time|method|                path|status|  size|            referrer|          user_agent|anomalous|     category|
+-------------+--------------------+------+--------------------+------+------+--------------------+--------------------+---------+-------------+
| 54.36.149.86|2019-01-22 03:59:...|   GET|/image/8739/speci...|   200|179495|https://www.zanbi...|Mozilla/5.0 (X11;...|        0|not anomalous|
| 5.62.206.249|2019-01-22 03:59:...|   GET|      /settings/logo|   302|159434|https://www.zanbi...|Mozilla/5.0 (Wind...|        0|not anomalous|
| 54.36.149.16|2019-01-22 03:59:...|   GET|/product/4031/20/...|   302| 74066|https://www.zanbi...|Dalvik/2.1.0 (Lin...|        0|not anomalous|
|66.249.66.195|2019-01-22 03:59:...|   GET|/image/65274/prod...|   302| 66812|https://www-zanbi...|Mozilla/5.0 (Linu...|        0|

In [64]:
!pip install numpy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


Pre-process and engineer features (might need to encode or use string indexer, depending on model)

In [65]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import unix_timestamp

training_df = training_df.withColumn(
    "unix_time", 
    unix_timestamp("time", "yyyy-MM-dd HH:mm:ss.SSS")
)

# Update VectorAssembler to include it
assembler = VectorAssembler(
    inputCols=["status", "size", "unix_time"] + [col + "_idx" for col in ["ip", "method", "path", "referrer", "user_agent"]],
    outputCol="features"
)

# note that REQUEST_TYPE = method
# note that REQUEST ARGUMENT = path 

# If doing classification, include label as well (anomalous or category)
label_indexer = StringIndexer(inputCol="anomalous", outputCol="label")  # for binary classification
# or: StringIndexer(inputCol="category", outputCol="label") for multi-class

# Combine steps
pipeline = Pipeline(stages=indexers + [assembler, label_indexer])
model = pipeline.fit(training_df)  # only if you're training
log_transformed = model.transform(training_df)
log_transformed.show(5)
log_transformed.select("features").show(truncate=False)

+-------------+--------------------+------+--------------------+------+------+--------------------+--------------------+---------+-------------+----------+------+----------+--------+------------+--------------+--------------------+-----+
|           ip|                time|method|                path|status|  size|            referrer|          user_agent|anomalous|     category| unix_time|ip_idx|method_idx|path_idx|referrer_idx|user_agent_idx|            features|label|
+-------------+--------------------+------+--------------------+------+------+--------------------+--------------------+---------+-------------+----------+------+----------+--------+------------+--------------+--------------------+-----+
| 54.36.149.86|2019-01-22 03:59:...|   GET|/image/8739/speci...|   200|179495|https://www.zanbi...|Mozilla/5.0 (X11;...|        0|not anomalous|1548129551|  79.0|       0.0|    59.0|        11.0|          25.0|[200.0,179495.0,1...|  0.0|
| 5.62.206.249|2019-01-22 03:59:...|   GET|     

Split data into test and train sets

Train the ML model(s) -- binary and multi-class?

save the pipeline and model 

there is a way to apply the ML model on a stream of data and output to console

might need to broadcast this data elsewhere so can show a UI..

In [66]:
train_data, test_data = log_transformed.randomSplit([0.8, 0.2], seed=42)


In [67]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    maxBins=1024  # or even 2048 if needed
)
rf_model = rf.fit(train_data)

In [68]:
predictions = rf_model.transform(test_data)

In [69]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print(f"AUC: {auc:.3f}")

AUC: 0.938


In [70]:
predictions.select("label", "prediction", "features").show(20)

+-----+----------+--------------------+
|label|prediction|            features|
+-----+----------+--------------------+
|  0.0|       0.0|[302.0,160997.0,1...|
|  0.0|       0.0|[301.0,11625.0,1....|
|  0.0|       0.0|[200.0,7899.0,1.5...|
|  0.0|       0.0|[304.0,45571.0,1....|
|  0.0|       0.0|[302.0,11337.0,1....|
|  0.0|       0.0|[302.0,157474.0,1...|
|  0.0|       0.0|[302.0,132633.0,1...|
|  0.0|       0.0|[200.0,118578.0,1...|
|  0.0|       0.0|[404.0,133588.0,1...|
|  0.0|       0.0|[200.0,4355.0,1.5...|
|  0.0|       0.0|[304.0,68827.0,1....|
|  0.0|       0.0|[200.0,118139.0,1...|
|  0.0|       0.0|[301.0,59802.0,1....|
|  0.0|       0.0|[302.0,84416.0,1....|
|  0.0|       0.0|[304.0,160726.0,1...|
|  0.0|       0.0|[200.0,42863.0,1....|
|  0.0|       0.0|[404.0,132667.0,1...|
|  0.0|       0.0|[404.0,146539.0,1...|
|  0.0|       0.0|[200.0,151087.0,1...|
|  0.0|       0.0|[302.0,32516.0,1....|
+-----+----------+--------------------+
only showing top 20 rows

